In [1]:
import pickle
import random
import os
import numpy as np
import tensorflow as tf
import numpy as np
import math
import pandas as pd 
import os

In [17]:
#
#   Corel converter
#

def corel_get_map(pdb, all_dist_paths, expected_l = -1):
    seqy = None
    
    mypath = ''
    for path in all_dist_paths:
        if os.path.exists(path + pdb + '-cb.npy'):
            mypath = path + pdb + '-cb.npy'
            #from where what are we reading here
            # ly length of the sequence, seqy is the sequence chain, cb_map is the distance mapping between pairs shape is (ly*ly)
            (ly, seqy, cb_map) = np.load(path + pdb + '-cb.npy', allow_pickle = True)
    if seqy == None:
        print('Expected distance map file for', pdb, 'not found at', all_dist_paths)
        exit(1)
    if 'cameo' not in mypath and expected_l > 0:
        assert expected_l == ly
        assert cb_map.shape == ((expected_l, expected_l))
    # Y contains the distance map
    Y = cb_map

    # Only CAMEO dataset has this issue
    if 'cameo' not in mypath:
        #if any value in Y is null then program stops
        assert not np.any(np.isnan(Y))
   #Cameo is in the path name then ignore
    if np.any(np.isnan(Y)):
        np.seterr(invalid='ignore')
        print('')
        print('WARNING!! Some values in the pdb structure of', pdb, 'l = ', ly, 'are missing or nan! Indices are: ', np.where(np.isnan(np.diagonal(Y))))
    Y[Y <= 8.0] = -1.0
    Y[Y>8]=0
    KK=Y

    for i in range (len(Y[0])):
        KK[i][i]=-(sum(Y[i])-Y[i][i])

    KK=KK.astype('float64')
    w , v = np.linalg.eig(KK)
    idx = w.argsort()
    w = w[idx]
    v = v[:,idx]

    u=np.zeros(shape=(len(v[1]), len(v[1])))
    for i in range (1,len(v)):

        u=u+(np.matmul(v[:,i][:,None],v[:,i][None,:]))/w[i]
    CC=u
    C=np.zeros(shape=np.shape(u))

    for i in range(len(CC[0])):
        for j in range(len(CC[0])):
            C[i][j]=CC[i][j]/np.sqrt(CC[i][i])/np.sqrt(CC[j][j])


    np.save("./data/corel_data_psicov/"+pdb + '-cb.npy', (ly, seqy, C), allow_pickle=True, fix_imports=True)





In [ ]:
from tqdm.notebook import tqdm
def load_list(file_lst, max_items = 1000000):
    if max_items < 0:
        max_items = 1000000
    protein_list = []
    f = open(file_lst, 'r')
    for l in f.readlines():
        protein_list.append(l.strip().split()[0])
    if (max_items < len(protein_list)):
        protein_list = protein_list[:max_items]
    return protein_list

dir_dataset               = './data/' 
deepcov_list = load_list(dir_dataset + 'psicov.lst')

all_dist_or_corel_paths = [dir_dataset + '/psicov/distance/'] #dir_dataset + '/psicov/distance/'

for i, pdb in tqdm(enumerate(deepcov_list)):
        #print(pdb)
        Y = corel_get_map(pdb, all_dist_or_corel_paths)
        

/Users/avyanshkatiyar/opt/miniconda3/lib/python3.9/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
